<a href="https://colab.research.google.com/github/pfmoro/VisualizacaoRefugiadosVenezuela/blob/main/Prototype_Venezuelan_Exports_and_GDP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from google.colab import files
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.offline import plot
import plotly.express as px
import os
from scipy.stats import spearmanr, linregress
from numpy.lib.stride_tricks import as_strided
from numpy.lib import pad
from plotly.subplots import make_subplots
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.stattools import adfuller

In [19]:
# Remove todos os arquivos CSV antigos
# Lista os arquivos atuais no ambiente de trabalho do Colab
files_in_colab = os.listdir('/content')

for file_name in files_in_colab:
    if file_name.endswith('.csv'):
        os.remove(file_name)
        print(f'Arquivo {file_name} removido.')

Arquivo Oil Refugees GDP.csv removido.
Arquivo Venezuelan Exports.csv removido.


In [9]:
uploaded = files.upload()

Saving Oil Refugees GDP.csv to Oil Refugees GDP.csv


In [10]:
file_name = 'Oil Refugees GDP.csv'

# Verifica se o arquivo foi carregado corretamente
if file_name in uploaded:
    # Lê o arquivo CSV usando a biblioteca Pandas
    df_oil_ref_gdp = pd.read_csv(file_name,sep=";",encoding='latin-1')
else:
    print(f"O arquivo '{file_name}' não foi encontrado.")

In [20]:
uploaded = files.upload()

Saving Venezuelan Exports.csv to Venezuelan Exports.csv


In [12]:
df_oil_ref_gdp = df_oil_ref_gdp.replace('#DIV/0!', pd.NaT if 'datetime' in df_oil_ref_gdp.columns else pd.NA)

In [13]:
df_oil_ref_gdp.head(5)

,Year,Refugees Venezuela,Refugees Venezuela Variation,refugees normalized,Refugees Venezuela Variation %,Crude Oil Year Average,Crude Oil Year Average Variation %,Crude Oil Year Average Variation % Lag +,Crude Oil Year Average Variation % Lag 2+,Crude Oil Year Average Variation % Lag 3+,Crude Oil Year Average Variation % Lag -,GDP,GDP Variation
0,2001.0,798.0,NaN,NaN,NaN,24.351825,-0.137,NaN,NaN,NaN,0.0237,300198.0,NaN
1,2002.0,1304.0,506.0,0.000168,0.63,24.927748,0.024,-0.1374,NaN,NaN,0.1593,277865.0,-0.0744
2,2003.0,2614.0,1310.0,0.000435,1.00,28.898903,0.159,0.0237,-0.1374,NaN,0.3057,261376.0,-0.0593
3,2004.0,3933.0,1319.0,0.000438,0.50,37.733388,0.306,0.1593,0.0237,-0.1374,0.4150,317486.0,0.2147
4,2005.0,5397.0,1464.0,0.000486,0.37,53.391025,0.415,0.3057,0.1593,0.0237,0.2041,361227.0,0.1378


In [14]:
df_oil_ref_gdp.columns

Index(['Year', 'Refugees Venezuela', 'Refugees Venezuela Variation ',
       'refugees normalized', 'Refugees Venezuela Variation %',
       'Crude Oil Year Average', 'Crude Oil Year Average Variation %',
       'Crude Oil Year Average Variation % Lag +',
       'Crude Oil Year Average Variation % Lag 2+',
       'Crude Oil Year Average Variation % Lag 3+',
       'Crude Oil Year Average Variation % Lag -', 'GDP', 'GDP Variation'],
      dtype='object')

In [21]:
file_name = 'Venezuelan Exports.csv'

# Verifica se o arquivo foi carregado corretamente
if file_name in uploaded:
    # Lê o arquivo CSV usando a biblioteca Pandas
    df_exports = pd.read_csv(file_name,sep=";",encoding='latin-1')
else:
    print(f"O arquivo '{file_name}' não foi encontrado.")

In [22]:
df_exports = df_exports.replace('#DIV/0!', pd.NaT if 'datetime' in df_exports.columns else pd.NA)

In [24]:
df_exports['Year']=df_exports['Year'].astype(int)

In [25]:
df_exports.head(5)

,Year,"Mineral fuels, mineral oils and products of their distillation",Other,Total,% Mineral Fuels,Crude Petroleum,Refined Petroleum,Refined perc
0,2000,2.659546e+10,5.194893e+09,3.179035e+10,0.84,1.827421e+10,7.319850e+09,0.29
1,2001,2.134751e+10,5.112868e+09,2.646038e+10,0.81,1.499250e+10,5.325348e+09,0.26
2,2002,2.019771e+10,5.522868e+09,2.572057e+10,0.79,1.748062e+10,1.662870e+09,0.09
3,2003,2.156093e+10,5.301000e+09,2.686193e+10,0.80,1.940519e+10,1.116019e+09,0.05
4,2004,2.946099e+10,7.821193e+09,3.728218e+10,0.79,1.966461e+10,8.587202e+09,0.30


In [18]:
df_exports.columns

Index(['Year',
       ' Mineral fuels, mineral oils and products of their distillation ',
       'Other', 'Total', '% Mineral Fuels', 'Crude Petroleum',
       'Refined Petroleum', 'Refined perc'],
      dtype='object')

Gráfico simples - Refugiados(Normalizado) x Ano

In [ ]:
fig = go.Figure([go.Scatter(x=df_oil_ref_gdp['Year'], y=df_oil_ref_gdp['refugees normalized'])])
fig.show()

In [ ]:
def rolling_spearman_slow(seqa, seqb, window):
    stridea = seqa.strides[0]
    ssa = as_strided(seqa, shape=[len(seqa) - window + 1, window], strides=[stridea, stridea])
    strideb = seqa.strides[0]
    ssb = as_strided(seqb, shape=[len(seqb) - window + 1, window], strides =[strideb, strideb])
    corrs = [spearmanr(a, b)[0] for (a,b) in zip(ssa, ssb)]
    return pad(corrs, (window - 1, 0), 'constant', constant_values=np.nan)


In [36]:
df_plot.tail(5)

,Year,GDP
18,2019.0,225730.0
19,2020.0,160123.0
20,2021.0,169108.0
21,2022.0,195658.0
22,2023.0,210817.0


In [41]:
#Remove NaN para plot:

var_x='Year'
var_y='GDP'
window=5

df_plot=df_oil_ref_gdp[[var_x, var_y]].dropna(how='any')
df_plot[var_x]=df_plot[var_x].astype(float)
df_plot[var_y]=df_plot[var_y].astype(float)*1e6


#Usaremos subplots:
fig = make_subplots(rows=2, cols=1)

#Cria um gráfico de dispersão para var_x
trace1 = go.Scatter(
    x=df_plot['Year'],
    y=df_plot[var_y].astype(float),
    mode='lines+markers',
    name=var_x
)

# Gráfico de Exportação venezuela
trace2 = go.Bar(
    x=df_exports['Year'],
    y=df_exports['Mineral fuels, mineral oils and products of their distillation'],
    name='Mineral Fuels',
)

trace3 = go.Bar(
    x=df_exports['Year'],
    y=df_exports['Other'],
    name='Other Products',
)

fig.update_layout(height=600, width=800,
    barmode='stack',
    title='Pauta de exportação venezuelana e PIB',
)

# Define os dados que serão plotados
data = [trace1, trace2, trace3]
fig.add_trace(trace1,row=2, col=1)
fig.add_trace(trace2,row=1, col=1)
fig.add_trace(trace3,row=1, col=1)


# Plota o gráfico no Google Colab
#plot(fig)
fig.show()
